##Machine Learning Assignment 3
##Author: Jaemin Lee
##Due: 11/10/19 

In [1]:
# mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Import libraries

In [2]:
import sklearn
import numpy as np
import pandas as pd
import nltk
nltk.download("stopwords")
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
set(stopwords.words('english'))
print()
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



# Read text files

In [0]:
train_txt = open("gdrive/My Drive/Machine Learning Assignment 3/reviews_train.txt", "r+")
test_txt = open("gdrive/My Drive/Machine Learning Assignment 3/reviews_test.txt", "r+")

#Function to clean the text document

In [0]:
# cleaning data
import re

REPLACE_NO_SPACE = re.compile("[,;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/?)|(\-)|(\/)")

def preprocess_reviews(reviews):
  reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
  reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]

  return reviews

# Cleaning training and test data (using regular expression operations)

In [0]:
# cleaning training data
reviews_train_clean = preprocess_reviews(train_txt)

# cleaning test data
reviews_test_clean = preprocess_reviews(test_txt)

In [0]:
reviews_train_clean

In [0]:
reviews_test_clean

# Removing stop words using NLTK package for review_train

In [0]:
from nltk.corpus import stopwords

english_stop_words = stopwords.words('english')
def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

no_stop_words_train = remove_stop_words(reviews_train_clean)


# Removing stop words using NLTK package for review_test

In [0]:
no_stop_words_test = remove_stop_words(reviews_test_clean)


# Vectorizing the word from the training and test data

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(binary = True)
vectorizer.fit(no_stop_words_train)

vector_train = vectorizer.transform(no_stop_words_train)
vector_test = vectorizer.transform(no_stop_words_test)
vector_test.shape


(25000, 85369)

# making true labels

In [10]:
positive = np.array([1])
positive = np.tile(positive, 12500)

negative = np.array([0])
negative = np.tile(negative, 12500)

positive = pd.DataFrame(positive)
#print(positive)
negative = pd.DataFrame(negative)
#print(negative)

true_labels = pd.DataFrame([])

true_labels = true_labels.append(positive)
true_labels = true_labels.append(negative)
true_labels.shape


(25000, 1)

# Apply KNN



In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

# find the optimal K value by plotting (set range from 20 to 50 bc accuracy went down after 50)
k_range = range(10,50)
scores = {}
scores_list = []

for k in k_range:
  knn = KNeighborsClassifier(n_neighbors=k*5)
  knn.fit(vector_train, true_labels)
  y_pred = knn.predict(vector_test)
  scores[k] = metrics.accuracy_score(true_labels, y_pred)
  scores_list.append(metrics.accuracy_score(true_labels, y_pred))

%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(k_range, scores_list)
plt.xlabel('Value of K for KNN')
plt.ylabel('Testing Accuracy')

In [21]:
# best value for k = 38
knn = KNeighborsClassifier(n_neighbors = 38)
knn.fit(vector_train, true_labels)
y_pred = knn.predict(vector_test)
print("Accuracy score for KNN is: " , accuracy_score(true_labels, y_pred))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


Accuracy score for KNN is:  0.67952


#Apply SVM

In [12]:
from sklearn import svm
from sklearn.metrics import accuracy_score

svm = svm.SVC(kernel = 'linear', probability=True)
prob = svm.fit(vector_train, true_labels).predict_proba(vector_test)
print("\nAccuracy score for SVM is: ", accuracy_score(true_labels, svm.predict(vector_test)))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Accuracy score for SVM is:  0.84712


In [0]:
pip install scikit-plot

# ROC Curve for KNN

In [0]:
import scikitplot as skplt
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

y_scores = knn.predict_proba(vector_test)
fpr, tpr, threshold = roc_curve(true_labels, y_scores[:, 1])
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC Curve of kNN')
plt.show()


# ROC Curve for SVM

In [0]:
import scikitplot as skplt
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

y_scores = svm.predict_proba(vector_test)
fpr, tpr, threshold = roc_curve(true_labels, y_scores[:, 1])
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC Curve of SVM')
plt.show()
